In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import json

## 1. Load Data

In [ ]:
try:
    df = pd.read_csv('startup_valuation_dataset.csv')
    print(f"Dataset loaded: {len(df)} records")
except FileNotFoundError:
    print("Error: Dataset not found.")
    raise

## 2. Define Target Variable - Valuation Success

In [ ]:
# Define "Success" as being in the top 25% of valuations
df['estimated_valuation_usd'] = df['estimated_valuation_usd'].fillna(0)
valuation_threshold = df['estimated_valuation_usd'].quantile(0.75)
print(f"Success Threshold: Valuation > ${valuation_threshold:,.0f}")

df['is_successful'] = (df['estimated_valuation_usd'] > valuation_threshold).astype(int)
print(f"\nClass distribution:")
print(df['is_successful'].value_counts())

## 3. Filter Training Data (Mature Companies)

In [ ]:
current_year = 2024
df['startup_age'] = current_year - df['founded_year']
training_df = df[df['startup_age'] >= 3].copy()
print(f"Training on {len(training_df)} companies (age >= 3 years)")

## 4. Feature Engineering

In [ ]:
# Funding features
training_df['funding_amount_usd'] = training_df['funding_amount_usd'].fillna(0)
training_df['log_funding'] = np.log1p(training_df['funding_amount_usd'])

# Revenue features
training_df['estimated_revenue_usd'] = training_df['estimated_revenue_usd'].fillna(0)
training_df['log_revenue'] = np.log1p(training_df['estimated_revenue_usd'])

# Efficiency metric
training_df['revenue_per_employee'] = training_df['estimated_revenue_usd'] / training_df['employee_count'].replace(0, 1)

# Investor features
training_df['investor_count'] = training_df['co_investors'].fillna('').apply(lambda x: x.count(',') + 1 if x != '' else 0)

# Round encoding
round_map = {'Pre-Seed': 1, 'Seed': 2, 'Angel': 2, 'Series A': 3, 'Series B': 4, 'Series C': 5, 'Series D': 6, 'IPO': 7}
training_df['round_encoded'] = training_df['funding_round'].map(round_map).fillna(0)

# Categorical encoding
features_to_encode = ['industry', 'region']
X_encoded = pd.get_dummies(training_df[features_to_encode], drop_first=True)

# Combine features
X = pd.concat([
    training_df[['log_funding', 'log_revenue', 'revenue_per_employee', 'employee_count', 'investor_count', 'round_encoded']],
    X_encoded
], axis=1)

y = training_df['is_successful']

print(f"Feature matrix shape: {X.shape}")
print(f"\nFeatures: {list(X.columns)}")

## 5. Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set: {len(X_train)} records")
print(f"Test set: {len(X_test)} records")

## 6. Train Random Forest Model

In [ ]:
print("Training Random Forest model...")

rf_model = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    min_samples_leaf=3,
    random_state=42
)

rf_model.fit(X_train, y_train)
print("Model training complete!")

## 7. Evaluate Model

In [ ]:
y_pred = rf_model.predict(X_test)

print("\n" + "="*40)
print("   VALUATION PREDICTION REPORT")
print("="*40)
print(classification_report(y_test, y_pred, target_names=['Standard', 'High Value (Top 25%)']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
print(f"\nOverall Accuracy: {accuracy:.2%}")

## 8. Feature Importance Analysis

In [ ]:
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n" + "="*40)
print("      WHAT DRIVES VALUATION?")
print("="*40)
print(feature_importances.head(10).to_string(index=False))

## 9. Save Model and Configuration

In [ ]:
# Save the trained model
joblib.dump(rf_model, 'startup_success_model.pkl')
print("Model saved as 'startup_success_model.pkl'")

# Save feature columns and configuration
model_config = {
    'feature_columns': list(X.columns),
    'valuation_threshold': float(valuation_threshold),
    'round_map': round_map,
    'current_year': current_year,
    'features_to_encode': features_to_encode
}

with open('model_config.json', 'w') as f:
    json.dump(model_config, f, indent=2)
print("Model configuration saved as 'model_config.json'")

print("\n✅ Model training and saving complete!")